In [1]:
# RUN THIS CELL
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# pandas tricks for better display
pd.options.display.max_columns = 50  
pd.options.display.max_rows = 500     
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

%matplotlib inline

In [2]:
colnames = np.loadtxt(('data/BYAREA_COUNTY.TXT'), dtype = str, delimiter = '|', max_rows = 1)
byarea_county = pd.DataFrame(np.loadtxt(('data/BYAREA_COUNTY.TXT'), dtype = str, delimiter = '|'), columns = colnames)[1:]

NameError: name 'data_dict' is not defined

In [41]:
# Source: https://data.cdc.gov/Health-Statistics/Social-Vulnerability-Index-2018-United-States-coun/48va-t53r
social_vulnerability = pd.read_csv("data/social_vulnerability_data.csv", dtype={'FIPS': 'str'})

In [34]:
byarea_county = byarea_county.replace("~", np.nan)
byarea_county = byarea_county.replace(".", np.nan)
byarea_county = byarea_county.replace("+", np.nan)
byarea_county = byarea_county.replace("-", np.nan)
byarea_county["FIPS"] = byarea_county["AREA"].str.replace(r'[^(]*\(|\)[^)]*', '').to_int()
county_aggregated = byarea_county.loc[(byarea_county['SITE'] == "All Cancer Sites Combined") &
                                      (byarea_county['RACE'] == "All Races") &
                                      (byarea_county['SEX'] == "Male and Female")]
county_aggregated_incidence = county_aggregated.loc[county_aggregated['EVENT_TYPE'] == "Incidence"]
county_aggregated_mortality = county_aggregated.loc[county_aggregated['EVENT_TYPE'] == "Mortality"]

/var/folders/zh/467jqd_55y773f1dwx6k_f440000gn/T/ipykernel_41337/374063700.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  byarea_county["FIPS"] = byarea_county["AREA"].str.replace(r'[^(]*\(|\)[^)]*', '').to_int()


AttributeError: 'Series' object has no attribute 'to_int'

In [43]:
social_predictors = [
    "FIPS",
    "E_TOTPOP", # population count
    "E_PCI", # per-capita income
    # "E_POV", # persons below poverty estimate
    # "E_UNEMP", # count of unemployed
    # "E_NOHSDP", # count non-high school grads
    # "E_DISABL", # count of disabled
    # "E_MINRTY", # count of minorities (all persons except white, non-Hispanic)
    # "E_MUNIT", # count of housing structures with 10+ units
    "EP_UNEMP", # unemployment rate
    "EP_NOHSDP", # no high school diploma rate
    "EP_DISABL", # disabled rate
    "EP_MINRTY", # minority rate
    "EP_MUNIT", # 10+ units rate
    "EP_POV" # share below poverty
]

social_vulnerability = social_vulnerability[social_predictors]

In [46]:
county_aggregated_enriched = county_aggregated.merge(social_vulnerability, on = "FIPS", how = "left")
county_aggregated_enriched.to_csv("enriched_county_data.csv")